# Importing Libraries and Data

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.pylab as pylab
import seaborn as sns
from IPython.display import display


%matplotlib inline

/Users/zhangxinzhou/anaconda/lib/python2.7/site-packages/IPython/html.py:14: ShimWarning: The `IPython.html` package has been deprecated. You should import from `notebook` instead. `IPython.html.widgets` has moved to `ipywidgets`.
  "`IPython.html.widgets` has moved to `ipywidgets`.", ShimWarning)


In [2]:
attributes_raw = pd.read_csv('attributes.csv')
product_desc_raw = pd.read_csv('product_descriptions.csv')
test_raw = pd.read_csv('test.csv')
train_raw = pd.read_csv('train.csv')

In [3]:
attributes_raw.head(20)

,product_uid,name,value
0,100001.0,Bullet01,Versatile connector for various 90° connection...
1,100001.0,Bullet02,Stronger than angled nailing or screw fastenin...
2,100001.0,Bullet03,Help ensure joints are consistently straight a...
3,100001.0,Bullet04,Dimensions: 3 in. x 3 in. x 1-1/2 in.
4,100001.0,Bullet05,Made from 12-Gauge steel
5,100001.0,Bullet06,Galvanized for extra corrosion resistance
6,100001.0,Bullet07,Install with 10d common nails or #9 x 1-1/2 in...
7,100001.0,Gauge,12
8,100001.0,Material,Galvanized Steel
9,100001.0,MFG Brand Name,Simpson Strong-Tie


In [4]:
attributes_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2044803 entries, 0 to 2044802
Data columns (total 3 columns):
product_uid    float64
name           object
value          object
dtypes: float64(1), object(2)
memory usage: 46.8+ MB


In [5]:
product_desc_raw.head(20)

,product_uid,product_description
0,100001,"Not only do angles make joints stronger, they ..."
1,100002,BEHR Premium Textured DECKOVER is an innovativ...
2,100003,Classic architecture meets contemporary design...
3,100004,The Grape Solar 265-Watt Polycrystalline PV So...
4,100005,Update your bathroom with the Delta Vero Singl...
5,100006,Achieving delicious results is almost effortle...
6,100007,The Quantum Adjustable 2-Light LED Black Emerg...
7,100008,The Teks #10 x 1-1/2 in. Zinc-Plated Steel Was...
8,100009,Get the House of Fara 3/4 in. x 3 in. x 8 ft. ...
9,100010,Valley View Industries Metal Stakes (4-Pack) a...


In [6]:
product_desc_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 124428 entries, 0 to 124427
Data columns (total 2 columns):
product_uid            124428 non-null int64
product_description    124428 non-null object
dtypes: int64(1), object(1)
memory usage: 1.9+ MB


In [7]:
train_raw.head(20)

,id,product_uid,product_title,search_term,relevance
0,2,100001,Simpson Strong-Tie 12-Gauge Angle,angle bracket,3.00
1,3,100001,Simpson Strong-Tie 12-Gauge Angle,l bracket,2.50
2,9,100002,BEHR Premium Textured DeckOver 1-gal. #SC-141 ...,deck over,3.00
3,16,100005,Delta Vero 1-Handle Shower Only Faucet Trim Ki...,rain shower head,2.33
4,17,100005,Delta Vero 1-Handle Shower Only Faucet Trim Ki...,shower only faucet,2.67
5,18,100006,Whirlpool 1.9 cu. ft. Over the Range Convectio...,convection otr,3.00
6,20,100006,Whirlpool 1.9 cu. ft. Over the Range Convectio...,microwave over stove,2.67
7,21,100006,Whirlpool 1.9 cu. ft. Over the Range Convectio...,microwaves,3.00
8,23,100007,Lithonia Lighting Quantum 2-Light Black LED Em...,emergency light,2.67
9,27,100009,House of Fara 3/4 in. x 3 in. x 8 ft. MDF Flut...,mdf 3/4,3.00


In [8]:
train_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 74067 entries, 0 to 74066
Data columns (total 5 columns):
id               74067 non-null int64
product_uid      74067 non-null int64
product_title    74067 non-null object
search_term      74067 non-null object
relevance        74067 non-null float64
dtypes: float64(1), int64(2), object(2)
memory usage: 2.8+ MB


In [9]:
test_raw.head(20)

,id,product_uid,product_title,search_term
0,1,100001,Simpson Strong-Tie 12-Gauge Angle,90 degree bracket
1,4,100001,Simpson Strong-Tie 12-Gauge Angle,metal l brackets
2,5,100001,Simpson Strong-Tie 12-Gauge Angle,simpson sku able
3,6,100001,Simpson Strong-Tie 12-Gauge Angle,simpson strong ties
4,7,100001,Simpson Strong-Tie 12-Gauge Angle,simpson strong tie hcc668
5,8,100001,Simpson Strong-Tie 12-Gauge Angle,wood connectors
6,10,100003,STERLING Ensemble 33-1/4 in. x 60 in. x 75-1/4...,bath and shower kit
7,11,100003,STERLING Ensemble 33-1/4 in. x 60 in. x 75-1/4...,bath drain kit
8,12,100003,STERLING Ensemble 33-1/4 in. x 60 in. x 75-1/4...,one piece tub shower
9,13,100004,Grape Solar 265-Watt Polycrystalline Solar Pan...,solar panel


# Data Wrangling

In [10]:
train_merged = train_raw.merge(product_desc_raw, on='product_uid', how='left')
train_merged.head(20)

,id,product_uid,product_title,search_term,relevance,product_description
0,2,100001,Simpson Strong-Tie 12-Gauge Angle,angle bracket,3.00,"Not only do angles make joints stronger, they ..."
1,3,100001,Simpson Strong-Tie 12-Gauge Angle,l bracket,2.50,"Not only do angles make joints stronger, they ..."
2,9,100002,BEHR Premium Textured DeckOver 1-gal. #SC-141 ...,deck over,3.00,BEHR Premium Textured DECKOVER is an innovativ...
3,16,100005,Delta Vero 1-Handle Shower Only Faucet Trim Ki...,rain shower head,2.33,Update your bathroom with the Delta Vero Singl...
4,17,100005,Delta Vero 1-Handle Shower Only Faucet Trim Ki...,shower only faucet,2.67,Update your bathroom with the Delta Vero Singl...
5,18,100006,Whirlpool 1.9 cu. ft. Over the Range Convectio...,convection otr,3.00,Achieving delicious results is almost effortle...
6,20,100006,Whirlpool 1.9 cu. ft. Over the Range Convectio...,microwave over stove,2.67,Achieving delicious results is almost effortle...
7,21,100006,Whirlpool 1.9 cu. ft. Over the Range Convectio...,microwaves,3.00,Achieving delicious results is almost effortle...
8,23,100007,Lithonia Lighting Quantum 2-Light Black LED Em...,emergency light,2.67,The Quantum Adjustable 2-Light LED Black Emerg...
9,27,100009,House of Fara 3/4 in. x 3 in. x 8 ft. MDF Flut...,mdf 3/4,3.00,Get the House of Fara 3/4 in. x 3 in. x 8 ft. ...


## try to find the similarity among the search term and the description and product title

### experiment on the single row:

In [187]:
texts = [train_merged.iloc[0,2].lower(), train_merged.iloc[0,3].lower(), train_merged.iloc[0,5].lower()]

#### extract the text, tokenize the sentences and clean up the text

In [188]:
texts

['simpson strong-tie 12-gauge angle',
 'angle bracket',
 'not only do angles make joints stronger, they also provide more consistent, straight corners. simpson strong-tie offers a wide variety of angles in various sizes and thicknesses to handle light-duty jobs or projects where a structural connection is needed. some can be bent (skewed) to match the project. for outdoor projects or those where moisture is present, use our zmax zinc-coated connectors, which provide extra resistance against corrosion (look for a "z" at the end of the model number).versatile connector for various 90 connections and home repair projectsstronger than angled nailing or screw fastening alonehelp ensure joints are consistently straight and strongdimensions: 3 in. x 3 in. x 1-1/2 in.made from 12-gauge steelgalvanized for extra corrosion resistanceinstall with 10d common nails or #9 x 1-1/2 in. strong-drive sd screws']

In [189]:
from nltk.tokenize import word_tokenize, RegexpTokenizer

In [190]:
#tokenize and remove punctuation
tokenizer = RegexpTokenizer(r'\w+')
texts_t = [tokenizer.tokenize(t) for t in texts]

In [191]:
#remove stopwords
from nltk.corpus import stopwords

def remove_stopwords(text):
    return [word for word in text if word not in stopwords.words('english')]
    
texts_t = map(remove_stopwords, texts_t)

In [192]:
#remove suffix of the words
from nltk.stem.wordnet import WordNetLemmatizer
def get_words_stem(tokenized_text):
    lemmatizer = WordNetLemmatizer()
    return map(lemmatizer.lemmatize, tokenized_text)

texts_t = map(get_words_stem, texts_t)

#### use freqdis() to check the frequencies of each word and compare it with the search term

In [193]:
from nltk import FreqDist

In [194]:
def get_freq_in_text(text, word):
#     print text
    freq = FreqDist(text)
    return freq[word]

In [195]:
for word in texts_t[1]:
    print word
    print 'freq in title: ', get_freq_in_text(texts_t[0], word)
    print 'freq in desc: ', get_freq_in_text(texts_t[2], word)

angle
freq in title:  1
freq in desc:  2
bracket
freq in title:  0
freq in desc:  0


#### use synsets module to check simlilarity

In [196]:
from nltk.corpus import wordnet as wn

In [202]:
kw = texts_t[1][0]
kw

'angle'

In [203]:
kw_syn = wn.synsets(kw)
kw_syn

[Synset('angle.n.01'),
 Synset('slant.n.01'),
 Synset('angle.n.03'),
 Synset('angle.v.01'),
 Synset('lean.v.01'),
 Synset('fish.v.01'),
 Synset('angle.v.04'),
 Synset('slant.v.02')]

* check similarity between search key word and the product title

In [204]:
def check_similarity_word_list(word_syn, words):
    result = {}
    # for each word in the text
    for w in words:
        syn_sims = []
        if len(wn.synsets(w)) > 0 :
#             print 'target word & synsets: ', w, wn.synsets(w)
            # for each synonym in the synsets of the word
            for syn in wn.synsets(w):
                # get the highest similarity between the synonyms of the reference word and that of the target word
                sim = max([ref_syn.path_similarity(syn) for ref_syn in word_syn if ref_syn.path_similarity(syn) > 0])
#                 print 'sim:', sim
                syn_sims.append(sim)
            print w, syn_sims    
            result[w] = max(syn_sims)
    return result

In [205]:
sim_kw_title = check_similarity_word_list(kw_syn, texts_t[0])
kw, sim_kw_title

simpson [0.125, 0.1]
strong [0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25]
tie [0.09090909090909091, 0.125, 0.1111111111111111, 0.125, 0.09090909090909091, 0.1, 0.08333333333333333, 0.09090909090909091, 0.1111111111111111, 0.14285714285714285, 0.16666666666666666, 0.1, 0.25, 0.16666666666666666, 0.14285714285714285, 0.14285714285714285, 0.125, 0.2]
12 [0.09090909090909091, 0.25]
gauge [0.09090909090909091, 0.1111111111111111, 0.1, 0.1, 0.1, 0.14285714285714285, 0.16666666666666666, 0.2, 0.1111111111111111, 0.14285714285714285, 0.14285714285714285]
angle [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]


('angle',
 {'12': 0.25,
  'angle': 1.0,
  'gauge': 0.2,
  'simpson': 0.125,
  'strong': 0.25,
  'tie': 0.25})

* check similarity between search key word and the product description

In [206]:
sim_kw_desc = check_similarity_word_list(kw_syn, texts_t[2])
kw, sim_kw_desc

angle [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]
make [0.09090909090909091, 0.0625, 0.25, 0.2, 0.25, 0.25, 0.2, 0.25, 0.16666666666666666, 0.16666666666666666, 0.2, 0.2, 0.16666666666666666, 0.2, 0.16666666666666666, 0.2, 0.16666666666666666, 0.2, 0.2, 0.2, 0.2, 0.2, 0.16666666666666666, 0.25, 0.2, 0.16666666666666666, 0.2, 0.14285714285714285, 0.1111111111111111, 0.2, 0.16666666666666666, 0.2, 0.2, 0.16666666666666666, 0.2, 0.2, 0.16666666666666666, 0.25, 0.2, 0.16666666666666666, 0.16666666666666666, 0.16666666666666666, 0.2, 0.14285714285714285, 0.125, 0.125, 0.2, 0.14285714285714285, 0.16666666666666666, 0.2, 0.16666666666666666]
joint [0.1111111111111111, 0.09090909090909091, 0.16666666666666666, 0.09090909090909091, 0.1, 0.08333333333333333, 0.14285714285714285, 0.14285714285714285, 0.14285714285714285, 0.16666666666666666, 0.25, 0.25, 0.25]
stronger [0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25, 0.25]
also [0.25]
provide [0.16666666666666666, 0.16666666666666666, 0.1, 0.16

('angle',
 {'1': 0.25,
  '12': 0.25,
  '2': 0.25,
  '3': 0.25,
  '9': 0.25,
  '90': 0.25,
  'also': 0.25,
  u'angle': 1.0,
  'angled': 1.0,
  'bent': 0.5,
  'coated': 0.25,
  'common': 0.25,
  'connection': 0.2,
  u'connector': 0.125,
  'consistent': 0.25,
  'consistently': 0.25,
  u'corner': 0.2,
  'corrosion': 0.1111111111111111,
  'drive': 0.3333333333333333,
  'duty': 0.1,
  'end': 0.25,
  'ensure': 0.25,
  'extra': 0.25,
  'fastening': 0.2,
  'gauge': 0.2,
  'handle': 0.2,
  'home': 0.25,
  u'job': 0.2,
  u'joint': 0.25,
  'light': 0.25,
  'look': 0.25,
  'made': 0.25,
  'make': 0.25,
  'match': 0.2,
  'model': 0.25,
  'moisture': 0.1111111111111111,
  u'nail': 0.2,
  'nailing': 0.2,
  'needed': 0.25,
  'number': 0.2,
  u'offer': 0.25,
  'outdoor': 0.25,
  'present': 0.25,
  u'project': 0.25,
  'provide': 0.16666666666666666,
  'repair': 0.3333333333333333,
  'resistance': 0.125,
  'screw': 0.16666666666666666,
  'sd': 0.1111111111111111,
  'simpson': 0.125,
  u'size': 0.25,
  'sk

* mean values of the similarities

In [207]:
def cal_similarities_mean(similarities_list):
    sims = [v for k,v in similarities_list.iteritems()]
    #drop na
    sims = np.array([e for e in sims if e != None])
    return sims.mean()

In [208]:
sim_kw_title_mean =cal_similarities_mean(sim_kw_title)
print sim_kw_title_mean

sim_kw_desc_mean = cal_similarities_mean(sim_kw_desc)
print sim_kw_desc_mean

0.345833333333
0.247348484848
